# In-class exercise 9: Deep Learning (Part 1A)
In this notebook we will see how to write efficient and numerically stable code.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time

%matplotlib inline

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import minmax_scale

In [2]:
X, y = load_breast_cancer(return_X_y=True)

# Scale each feature to [-1, 1] range
X = minmax_scale(X, feature_range=(-1, 1))

In [3]:
X.shape,y.shape

((569, 30), (569,))

# 1. Vectorization 向量化

## 1.1. Logistic regression (two classes)

**Setting:** Logistic regression (two classes)

**Task:** Generate predictions for the entire dataset

In [4]:
n_features = X.shape[1]
w = np.random.normal(size=[n_features], scale=0.1)  # weight vector 30个数
b = np.random.normal(size=[1])  # bias

In [5]:
def sigmoid(t):
    """Apply sigmoid to the input array."""
    return 1 / (1 + np.exp(-t))

### Bad - for loops

In [7]:
def predict_for_loop(X, w, b):
    """Generate predictions with a logistic regression model using a for-loop.
    
    Args:
        X: data matrix, shape (N, D)
        w: weights vector, shape (D)
        b: bias term, shape (1)
        
    Returns:
        y: probabilies of the positive class, shape (N)
    """
    n_samples = X.shape[0]
    y = np.zeros([n_samples])
    for i in range(n_samples):      #因为有for loop所以很慢
        score = np.dot(X[i], w) + b
        y[i] = sigmoid(score)       #activation 
    return y

### Good - vectorization

In [6]:
def predict_vectorized(X, w, b):
    """Generate predictions with a logistic regression model using vectorized operations.
    
    Args:
        X: data matrix, shape (N, D)
        w: weights vector, shape (D)
        b: bias term, shape (1)
        
    Returns:
        y: probabilies of the positive class, shape (N)
    """
    scores = X @ w + b           #@符号相当于np.dot(X,w),此处把X中的每行看成一体的，把X和w直接做矩阵乘法
    y = sigmoid(scores)
    return y

### Compare the runtime of two variants

In [7]:
%%timeit
predict_for_loop(X, w, b)

3.91 ms ± 27.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%%timeit
predict_vectorized(X, w, b)

28.5 µs ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## 1.2. K-nearest neighbors
A more complicated task: compute the matrix of pairwise distances.

Given a data matrix `X` of size `[N, D]`, compute the matrix `dist` of pairwise distances of size `[N, N]`, where `dist[i, j] = l2_distance(X[i], X[j])`.

### Bad - for loops

In [8]:
def l2_distance(x, y):
    """Compute Euclidean distance between two vectors."""
    return np.sqrt(np.sum((x - y) ** 2))  #element-wise

In [9]:
def distances_for_loop(X):
    """Compute pairwise distances between all instances (for loop version).
    
    Args:
        X: data matrix, shape (N, D)
        
    Returns:
        dist: matrix of pairwise distances, shape (N, N)
    """
    n_samples = X.shape[0]
    distances = np.zeros([n_samples, n_samples])
    for i in range(n_samples):
        for j in range(n_samples):
            distances[i, j] = l2_distance(X[i], X[j])
    return distances

In [10]:
dist1 = distances_for_loop(X)

### Good - vectorization

How can we compute all the distances in a vectorized way?

Start with a simpler example.

In [12]:
x = np.arange(5, dtype=np.float64)
print(x)

[0. 1. 2. 3. 4.]


In [18]:
x.reshape([-1,1])   #让x 变为[:,1]的形式

array([[0.],
       [1.],
       [2.],
       [3.],
       [4.]])

In [19]:
x.reshape([1,-1])  #让x变为[1,:]的形式

array([[0., 1., 2., 3., 4.]])

In [20]:
x.reshape([-1,1])-x.reshape([1,-1])  #numpy会自动扩充（相当于使用了repeat）
#vector形状 [5,1]-[1,5]

array([[ 0., -1., -2., -3., -4.],
       [ 1.,  0., -1., -2., -3.],
       [ 2.,  1.,  0., -1., -2.],
       [ 3.,  2.,  1.,  0., -1.],
       [ 4.,  3.,  2.,  1.,  0.]])

In [21]:
x.reshape([-1,1]) .repeat(5,axis =1)  #在列的方向上重复5次

array([[0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1.],
       [2., 2., 2., 2., 2.],
       [3., 3., 3., 3., 3.],
       [4., 4., 4., 4., 4.]])

Increase the dimension of an array using `np.newaxis`

In [14]:
x[:, np.newaxis] #相当于x.reshape([-1,1])或x[:,None]，加了一个dimension进去，从[5]变为[5,1]

array([[0.],
       [1.],
       [2.],
       [3.],
       [4.]])

In [22]:
x[np.newaxis, :]

array([[0., 1., 2., 3., 4.]])

In [14]:
x[np.newaxis, :] - x[:, np.newaxis]

array([[ 0.,  1.,  2.,  3.,  4.],
       [-1.,  0.,  1.,  2.,  3.],
       [-2., -1.,  0.,  1.,  2.],
       [-3., -2., -1.,  0.,  1.],
       [-4., -3., -2., -1.,  0.]])

In [23]:
x[None,:] - x[:,None]

array([[ 0.,  1.,  2.,  3.,  4.],
       [-1.,  0.,  1.,  2.,  3.],
       [-2., -1.,  0.,  1.,  2.],
       [-3., -2., -1.,  0.,  1.],
       [-4., -3., -2., -1.,  0.]])

In [35]:
def distances_vectorized(X):
    """Compute pairwise distances between all instances (vectorized version).
    
    Args:
        X: data matrix, shape (N, D)   #X不是一个vector而是一个matrix
        
    Returns:
        dist: matrix of pairwise distances, shape (N, N)
    """
    return np.sqrt(((X[:, None, :] - X[None, :, :])**2).sum(axis =-1)) #第一个None扩充为569，第二个也扩充为569
#sum 里的-1也可以写2,表示把axis=2的维度加起来即把[569,569,30]变为[569.569]

In [36]:
dist2 = distances_vectorized(X)

Make sure that both variants produce the same results

In [37]:
# Direct comparison fails because of tiny numerical differences
np.all(dist1 == dist2)

True

In [38]:
# Two results are very close
np.linalg.norm(dist1 - dist2, ord='fro')

0.0

In [34]:
# Use np.allclose to compare
np.allclose(dist1, dist2)

True

### Best - library function

In [41]:
from scipy.spatial.distance import cdist, pdist, squareform

dist3 = cdist(X, X)
dist4 = squareform(pdist(X))    #两种方法结果一样，都是计算L2

### Compare the runtime

In [42]:
%%timeit
dist1 = distances_for_loop(X)

1.95 s ± 33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
%%timeit
dist2 = distances_vectorized(X)

32.5 ms ± 189 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [43]:
%%timeit
dist3 = cdist(X, X)

3.33 ms ± 160 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [44]:
%%timeit
dist4 = squareform(pdist(X))

1.93 ms ± 46.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Lessons:
1. For-loops are extremely slow! Avoid them whenever possible.
2. A better alternative - use matrix operations & broadcasting
3. An even better alternative - use library functions (if they are available).
4. Implementations with for-loops can be useful for debugging vectorized code.

# 2. Numerical stability
Typically, GPUs use single precision (32bit) floating point numbers (in some cases even half precision / 16bit). This significantly speeds ups the computations, but also makes numerical issues a lot more likely. 
Because of this we always have to be extremely careful to implement our code in a numerically stable way.

Most commonly, numerical issues occur when dealing with `log` and `exp` functions (e.g. when computing cross-entropy of a categorical distribution) and `sqrt` for values close to zero (e.g. when computing standard deviations or normalizing the $L_2$ norm).

## 2.1. Avoiding numerical overflow (exploding `exp`)

Softmax function $f : \mathbb{R}^D \to \Delta^{D - 1}$ converts a vector $\mathbf{x} \in \mathbb{R}^D$ into a vector of probabilities.

$$f(\mathbf{x})_j = \frac{\exp(x_j)}{\sum_{d=1}^{D} \exp(x_d)}$$

Apply the softmax function to the following vector.

In [10]:
x = np.linspace(0., 4., 5).astype(np.float32)
x

array([0., 1., 2., 3., 4.], dtype=float32)

In [11]:
x_shifted = x - np.max(x)
denominator = np.exp(x_shifted).sum()
np.exp(x_shifted) / denominator        #验证加上a之后结果确实一样

array([0.01165623, 0.03168492, 0.08612854, 0.23412165, 0.6364086 ],
      dtype=float32)

In [4]:
# Our code here
denominator = np.exp(x).sum()
np.exp(x) / denominator

array([0.01165623, 0.03168492, 0.08612854, 0.23412167, 0.6364086 ],
      dtype=float32)

Now apply it to the following vector

In [5]:
x = np.linspace(50., 90., 5).astype(np.float32)
x

array([50., 60., 70., 80., 90.], dtype=float32)

In [8]:
np.exp(x)  #最多只能到e+38,exp(90)超出了

/home/melina/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in exp
  """Entry point for launching an IPython kernel.


array([5.1847055e+21, 1.1420073e+26, 2.5154387e+30, 5.5406225e+34,
                 inf], dtype=float32)

In [9]:
# Our code here
denominator = np.exp(x).sum()
np.exp(x) / denominator

/home/melina/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/home/melina/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/home/melina/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


array([ 0.,  0.,  0.,  0., nan], dtype=float32)

How to avoid the exposion?

In [29]:
x_shifted = x - np.max(x)
denominator = np.exp(x_shifted).sum()
np.exp(x_shifted) / denominator

array([4.248161e-18, 9.357198e-14, 2.061060e-09, 4.539787e-05,
       9.999546e-01], dtype=float32)

## 2.2. Working in the log-space / simplifying the expressions

Binary cross entropy (BCE) loss for a logistic regression model (corresponds to negative log-likelihood of a Bernoulli model)

$$\log p(\mathbf{y} \mid \mathbf{X}, \mathbf{w}, b) = -\sum_{i=1}^{N} y_i \log \sigma(\mathbf{w}^T \mathbf{x}_i + b) + (1 - y_i) \log (1 - \sigma(\mathbf{w}^T \mathbf{x}_i + b))$$


Implement the BCE computation.

In [12]:
# TODO
def sigmoid(t):
    return 1 / (1 + np.exp(-t))

def binary_cross_entropy_unstable(scores, labels):
    return -labels * np.log(sigmoid(scores)) - (1 - labels) * np.log(1 - sigmoid(scores)) #对于一个single sample

In [14]:
x = np.array([[20., 20.]]) #shape[1,2]
w = np.array([[1., 1.]])   #shape[2,1]
y = np.array([1.])

scores = x @ w.T
binary_cross_entropy_unstable(scores, y)   #sigmoid(40) = 1, 1-sigmoid(40) = 0, log(0) = Inf

/home/melina/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  
/home/melina/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in multiply
  


array([[nan]])

Try to simplify the BCE loss as much as possible

In [15]:
# TODO
def binary_cross_entropy_stable(scores, labels):
    return np.log(1 + np.exp(scores)) - labels * scores    #证明过程在IN-CLASS06 Problem2

binary_cross_entropy_stable(scores, y)

array([[0.]])

## 2.3. Loss of numerical precision

Implement the log sigmoid function 

$$f(x) = \log \sigma(x) = \log \left(\frac{1}{1 + \exp(-x)}\right) = -\log(1+\exp(-x))$$

In [16]:
# Your code here
def log_sigmoid_unstable(x):
    return np.log(1 / (1 + np.exp(-x)))

`float32` has much lower "resolution" than `float64`

In [19]:
x = np.linspace(0, 30, 11).astype(np.float32)
print(x)
log_sigmoid_unstable(x)         #x大，exp(-x)很小 log(1)=0

[ 0.  3.  6.  9. 12. 15. 18. 21. 24. 27. 30.]


array([-6.9314718e-01, -4.8587341e-02, -2.4756414e-03, -1.2338923e-04,
       -6.1989022e-06, -3.5762793e-07,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00], dtype=float32)

In [20]:
x = np.linspace(0, 30, 11).astype(np.float64)
log_sigmoid_unstable(x)

array([-6.93147181e-01, -4.85873516e-02, -2.47568514e-03, -1.23402190e-04,
       -6.14419348e-06, -3.05902274e-07, -1.52299796e-08, -7.58256125e-10,
       -3.77513576e-11, -1.87960758e-12, -9.34807787e-14])

Implement the log-sigmoid function in a numerically stable way

In [25]:
def log_sigmoid_stable(x):
    return -np.log1p(np.exp(-x)) # =np.log(1+np.exp(-x)) 可以用float32得到精确的答案

In [26]:
x = np.linspace(0, 30, 11).astype(np.float32)
log_sigmoid_stable(x)

array([-6.9314718e-01, -4.8587352e-02, -2.4756850e-03, -1.2340219e-04,
       -6.1441933e-06, -3.0590226e-07, -1.5229981e-08, -7.5825601e-10,
       -3.7751344e-11, -1.8795287e-12, -9.3576229e-14], dtype=float32)

Relevant functions: `np.log1p`, `np.expm1`, `scipy.special.logsumexp`, `scipy.special.softmax` -- these are also implemented in all major deep learning frameworks.

In [28]:
#np.expm1(x) = np.exp(x) - 1

## Lessons:
1. Be especially careful when working with `log` and `exp` functions in **single precision** floating point arithmetics
2. Work in the log-space when possible
3. Use numerically stable library functions when available